In [1]:
import cv2
import os
from math import ceil
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import ipynb.fs.defs.FeatureMatching as FeatureMatching
import ipynb.fs.defs.GraphBuilding as GraphBuilding
import ipynb.fs.defs.SimpleGraphStitching as SimpleGraphStitching
import ipynb.fs.defs.Utils as Utils
from IPython.display import display
import IPython

In [2]:
BENCHMARK_DIR = "benchmark"
FILENAME = "results.csv"
INDEX = ["number_of_matches", "noise_std"]

In [3]:
def compute_error(H_gt, H):
    return np.linalg.norm(np.eye(3)-H_gt@np.linalg.inv(H))

In [4]:
def compute_avg_error(H_gt, H):
    return np.mean([compute_error(H_gt_node, H_node) for H_gt_node, H_node in zip(H_gt,H)])

In [5]:
def compute_ground_truth(dataset_name,imgs,T_norm,matching_threshold,matches_th, idx_ref):
    matches_dict_gt, _ = FeatureMatching.get_feature_matches(dataset_name,
                        imgs,
                        T_norm,
                        matching_threshold = matching_threshold,
                        number_of_matches = 1,
                        matches_th = matches_th,
                        RANSACmaxIters = 2000,
                        save_output = False,
                        save_images = False,
                        noisy_matching = False,
                        verbose = False
                       )

    M_gt, _, _ = GraphBuilding.build_simple_graph_matrices(dataset_name,
                imgs,
                matches_dict_gt,
                verbose = False,
                save_output = False
               )

    H_gt, stitched_image_gt = SimpleGraphStitching.simple_graph_stitching(dataset_name,
                                imgs,
                                T_norm,
                                M_gt, 
                                idx_ref = idx_ref,
                                verbose = False,
                                save_output = False,
                                beautify = True,
                                warp_shape = [10000,10000])
    
    return H_gt, stitched_image_gt

In [6]:
def visualize_stats(df):
    df.sort_values(by=INDEX, ascending=True, inplace=True)
    display(df)
    
    df_v = df.drop(columns="Experiments")
    
    noises = np.unique([ v[1] for v in df_v.index.values]) #noises
    matches = np.unique([ v[0] for v in df_v.index.values]) #matches

    Utils.scatter_with_slider(noises, df_v.reorder_levels(['noise_std','number_of_matches']).sort_index(), "Number of matches", "Noise std")
    Utils.scatter_with_slider(matches, df_v, "Noise std", "Number of matches")
    Utils.mesh_plot(df_v)


In [7]:
def visualize_results(df, results, stitched_img_gt):
    
    Utils.plot_images(results, stitched_img_gt)
    
    visualize_stats(df)
    

In [8]:
def load_gt_and_stats(dataset_dir,
                df_file,
                dataset_name,
                imgs,
                T_norm,
                matching_threshold,
                matches_th, 
                idx_ref,
                results):
    index = INDEX
    gt_file = os.path.join(dataset_dir,"gt.npy")
    gt_img_file = os.path.join(dataset_dir,"gt_img.jpg")
    
    #Compute GT if not existing
    if os.path.isfile(gt_file):
        H_gt = np.load(gt_file)
        stitched_img_gt = cv2.cvtColor(cv2.imread(gt_img_file), cv2.COLOR_BGR2RGB)
        df = pd.read_csv(df_file, index_col = index)
    else:
        H_gt, stitched_img_gt = compute_ground_truth(dataset_name,imgs,T_norm,matching_threshold,matches_th, idx_ref)
        cv2.imwrite(gt_img_file, cv2.cvtColor(stitched_img_gt, cv2.COLOR_RGB2BGR))
        np.save(gt_file,H_gt)
        columns = index + [r["name"] for r in results] + ["Experiments"]
        df = pd.DataFrame(columns=columns)
        df.set_index(index,inplace=True)
        
    return H_gt, stitched_img_gt, df

In [9]:
def run_benchmark(dataset_name,
                  imgs,
                  T_norm,
                  matching_threshold,
                  matches_th, 
                  idx_ref, 
                  results, 
                  number_of_matches, 
                  noise_std, 
                  visualize = True):
    
    benchmark_dir = BENCHMARK_DIR
    filename = FILENAME
    
    dataset_dir = os.path.join(benchmark_dir,f"{dataset_name}_{idx_ref}_{matching_threshold}_{matches_th}")
  
    df_file = os.path.join(dataset_dir, filename)
    os.makedirs(dataset_dir, exist_ok = True)
    
    H_gt, stitched_img_gt, df = load_gt_and_stats(dataset_dir,
                df_file,
                dataset_name,
                imgs,
                T_norm,
                matching_threshold,
                matches_th, 
                idx_ref,
                results)
    
    index = (number_of_matches, noise_std)
    if index in df.index:
        exp = df.loc[index,'Experiments'] + 1
        new_row = [df.loc[index,r["name"]] * ((exp-1)/exp) + compute_avg_error(H_gt,r["H"])*1/exp for r in results]
        new_row.append(exp)
    else:
        new_row = [compute_avg_error(H_gt,r["H"]) for r in results]
        new_row.append(1)
    
    df.loc[index,:] = new_row
    df.to_csv(df_file)
    
    if visualize:
        visualize_results(df, results, stitched_img_gt)


## Visualize Statistics
It is useful to evaluate performances of methods in a dataset without computing a new benchmark.

In [10]:
def dropdown_stats_eventhandler(change):
    with output:
        selected = change.new
        file = os.path.join(BENCHMARK_DIR, os.path.join(selected, FILENAME))
        df = pd.read_csv(file, index_col = INDEX)

        visualize_stats(df)
        IPython.display.clear_output(wait=True)

In [11]:
directories = os.listdir(BENCHMARK_DIR)
output = widgets.Output()
dropdown_stats = widgets.Dropdown(options = directories, value=None, description='Benchmarks:')
dropdown_stats.observe(dropdown_stats_eventhandler, names='value') 
display(dropdown_stats, output)
IPython.display.clear_output(wait=True)

Dropdown(description='Benchmarks:', options=('sun_2_0.7_30',), value=None)

Output()